In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect = True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Station = Base.classes.station
Measurement = Base.classes.measurement

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Identify the variables within the data set.
measurement_first = session.query(Measurement).first()

measurement_first.__dict__

In [ ]:
# Find the most recent date in the data set.
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database, calculate the date one year from the last date in data set.
last_date = dt.date(2017, 8 ,23)

first_date = last_date - dt.timedelta(days = 365)
first_date

In [ ]:
# Perform a query to retrieve the data and precipitation scores
prcp_date = session.query(Measurement.date, Measurement.prcp)\
    .filter(Measurement.date >= first_date, Measurement.date <= last_date).all()

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
prcp_date_df = pd.DataFrame(prcp_date)

In [ ]:
# Sort the dataframe by date
prcp_date_sorted = prcp_date_df.sort_values(by = "date")

prcp_date_sorted

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
prcp_date_sorted.plot(x = "date", y = "prcp")

plt.xlabel("Date")
plt.ylabel("Temperature")
plt.xticks(rotation = 90)

plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_date_sorted.describe()

# Exploratory Station Analysis

In [ ]:
# Identify the variables within the data set
station_first = session.query(Station).first()

station_first.__dict__

In [ ]:
# Design a query to calculate the total number stations in the dataset
session.query(Station.name).distinct().count()

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
stations = [Station.name, func.count(Measurement.station).label('counts'), Station.id, Station.station]
stations_grouped = session.query(* stations).filter(Station.station == Measurement.station)\
        .group_by(Station.station).order_by(func.count(Measurement.station).label('counts').desc()).all()

stations_grouped

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
waihee = session.query(Measurement.station, Measurement.tobs).filter(Measurement.station == 'USC00519281').all()

waihee_df = pd.DataFrame(waihee).set_index("station")

waihee_df.describe().loc[["min", "mean", "max"]]

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
waihee_lastyr = session.query(Measurement.station, Measurement.tobs)\
    .filter(Measurement.station == 'USC00519281', Measurement.date >= first_date, Measurement.date <= last_date).all()

waihee_lastyr_df = pd.DataFrame(waihee_lastyr).set_index("station")

waihee_lastyr_df.head()

In [ ]:
waihee_plot = waihee_lastyr_df.plot.hist(bins = 12)
waihee_plot.set_xlabel("Temperature")

plt.show()

# Close session

In [ ]:
# Close Session
session.close()